In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ast
import os
from pathlib import Path
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

In [ ]:
tmp = Path("/tmp")
dummy_package = tmp / "dummy_package"
dummy_package.mkdir(parents=True, exist_ok=True)

init = dummy_package / "__init__.py"

init_text = """from .attrape import hello


def aloha():
    hello()
"""
init.write_text(init_text)

attrape_text = """import os

def hello():
    print("Hello, World!")
"""

attrape_file = dummy_package / "attrape.py"
attrape_file.write_text(attrape_text)

In [ ]:
class CodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.graph = nx.DiGraph()
        self.current_function = None
        self.current_file = None

    def visit_FunctionDef(self, node):
        func_name = node.name
        self.current_function = func_name
        # Add the node with the file attribute
        self.graph.add_node(func_name, file=self.current_file)
        self.generic_visit(node)
        self.current_function = None

    def visit_Call(self, node):
        if isinstance(node.func, ast.Attribute):
            called_func = node.func.attr
        elif isinstance(node.func, ast.Name):
            called_func = node.func.id
        else:
            called_func = None

        if self.current_function and called_func:
            self.graph.add_edge(self.current_function, called_func)

        self.generic_visit(node)


def analyze_code(base_path):
    analyzer = CodeAnalyzer()
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(".py"):
                analyzer.current_file = file  # Set the current file
                with open(os.path.join(root, file), "r") as f:
                    tree = ast.parse(f.read())
                    analyzer.visit(tree)
    return analyzer.graph


def visualize_graph(graph):
    # Use Graphviz for layout
    plt.figure(figsize=(20, 20))
    pos = graphviz_layout(
        graph, prog="dot"
    )  # 'dot' is one of the Graphviz layout algorithms

    # Draw the nodes and the black labels
    nx.draw(graph, pos, with_labels=True, node_color="lightblue", font_weight="bold")

    # Offset for red labels
    label_pos = {
        key: (value[0], value[1] - 20) for key, value in pos.items()
    }  # Adjust the offset as needed

    # Get the file labels and draw them in red below the nodes
    labels = nx.get_node_attributes(graph, "file")
    nx.draw_networkx_labels(graph, label_pos, labels, font_size=10, font_color="red")

    plt.show()


# Usage Example
# Replace 'path_to_repository' with the actual path of your Python repository
graph = analyze_code(tmp / "dummy_package")
visualize_graph(graph)

In [ ]:
import pandas as pd

pd.Series(nx.centrality.degree_centrality(graph)).sort_values()